## What is the future of 'chatgpt like'AI models ?

![Stable_diffusion_v2_generated][chatbots]

* Chatgpt is the latest popular tech buzz around the block

* It was published in singapore's [mainstream paper](https://www.straitstimes.com/tech/civil-servants-to-soon-use-chatgpt-to-help-with-research-speech-writing) that all civil servants will be using chatgpt for writing reports & speeches 

* It was also mentioned in [parliament](https://www.moe.gov.sg/news/parliamentary-replies/20230206-artificial-intelligence-technologies-chatgpt) by Education Minister Chan Chun Sing quote "... artificial intelligence (AI) tool ChatGPT could be a useful tool for learning ... when students have mastered basic concepts and thinking skills" unquote

Chatgpt in essence is a Large Language Model (LLM) that is built for a dialogue setting (hence the prefix chat) by using reinforcement learning techniques with human feedback loops. In simple english it is self-taught to learn natural langauge with human feedback

Its parameters (or the number of internal representational variables in layman terms) are massive at 175 billion, hence the label Large Language Model.

LLMs are often cited as foundational models because they are the basic building block for various Natural Language Understanding applications, of which I'm evaluating and exploiting for this post.

Currently due to the massiveness of the model, it is almost difficult or impossible for users to run such models on consumer-grade desktops or laptops. These models require specialized AI hardware accelerators (GPUs or TPUs) in order be to performant. The bigger models will require larger memory specs in order to keep track of the billions of parameters.

I'm grateful and fortunate that I undertook training under AI Singapore's AI apprenticeship programme to be exposed to developing and operating these complex AI models for real-world applications at scale. Allowing me to put together this basic demonstration of the potential of LLM as a useful toolkit.

[chatbots]: https://wykeith.github.io/images/google-t5-b.png

In [2]:
# Setup + configs for lambdacloud instance

lambdacloud = backend(
    client_name = "huggingface",
    client_connection = "http://localhost:5000"
)
print("LLM model type:\n\t",lambdacloud.client.get_model_params())
print("LLM model arguments:\n\t",lambdacloud.client.get_model_inputs())

llm = create_model(client=lambdacloud, llm_kwargs={"temperature": 0.2, "max_tokens": 4096})

LLM model type:
	 {'model_name': 'google/flan-t5-xxl', 'model_path': 'google/flan-t5-xxl'}
LLM model arguments:
	 ['temperature', 'max_tokens', 'n', 'top_p', 'top_k', 'repetition_penalty', 'do_sample', 'client_timeout']


In [3]:
# PSLE passage extracted from 
# https://static1.squarespace.com/static/5a8433dffe54efed2118456f/t/60ab98e365ffd53bbdce87af/1621858531970/PSLE+English+OE+Comprehension+-+Questions+Only.pdf

PSLE_passage = """
        “Jinhang! How many times have I told you not to use the computer
        for so long? Do you want thicker lenses for your glasses?” Mother’s voice
        shot out from the kitchen.
        I got out of my seat and walked into the kitchen. Mother was
        making dumplings.
        I sat down beside her and picked up some dough. Seeing this, Mother
        smiled at me, “Yes, that’s a good girl.”
        I started wrapping, finishing each dumpling off with a clumsy pinch.
        After I had made ten dumplings, I looked at my handiwork, dissatisfied.
        Picking up my tray of dumplings, I walked towards the wastepaper basket.
        Mother stopped me, saying, “They don’t look like much, but they will be
        tasty nevertheless.” I sat back down, then tried to continue making more
        dumplings, but I found myself unconsciously looking at the ugly ones I had
        made earlier.
        “Do you remember when you were younger? You were always sick,”
        Mother started. “At school, the other kids kicked you, and snatched things
        from you, but you never fought back.” I winced at the memory.
        Growing up, I was a timid child. Once I had taken my favourite toy
        lion to kindergarten. Everyone swarmed around me, admiring it. I loved
        the attention. That afternoon, my classmate grabbed the lion, claiming it
        as hers. When I finally summoned the courage to tell Mother about this, I
        expected her to comfort me. Instead, she gave me one of the worst
        telling-offs of my life.
        As if reading my thoughts, Mother continued, “Do you know why I
        reprimanded you? You did nothing wrong. Father and I knew you were
        vulnerable, and needed you to learn to defend yourself.” Then it hit me.
        All these years, I thought I had been scolded because I was useless and
        unable to protect my belongings, but all Mother wanted was to teach me
        to stand up for myself.
        “Now look carefully at this big ball of dough,” Mother said, pointing
        to the bowl. “That used to be me. Then I decided to sacrifice part of me
        for you.” She plucked off a small piece of dough and flattened it with a
        rolling pin.
        “Now examine this wrapper.” She held up the thin sheet of dough.
        “This was you when you were born. At that time even a squeeze could
        scar you for life.” Mother loaded a tablespoon of filling into the centre of
        the wrapper. “Then we started added in filling: carrots, vegetables, and
        mushrooms. We taught you values and equipped you with all the life skills
        you needed.”
        “As a last move, we sealed our gift from God - with love.” Mother
        pinched the dumpling close. I smiled, knowing that I now had to be
        independent.
        """
with open("psle.txt","w") as f:
    f.write(PSLE_passage)

In [4]:
# Summarization task

text_splitter = tokenizer()
prompt_template = """Write a concise summary of the following:

{text}

CONCISE SUMMARY:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
summarizer = reduceinputs.from_params(llm, prompt, text_splitter)
summarizer.run(PSLE_passage)

'Jinhang was a timid child. When she was scolded by her mother, she thought she was useless and unable to protect her belongings. But all these years, Mother wanted to teach her to stand up for herself.'

In [5]:
prompt_template = """Given the following document, provide a final answer. 
QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["summaries", "question"])
answerqns = qainputs(llm, chain_type="stuff", prompt=prompt)
docs = readdoc("psle.txt").load()

In [6]:
# QA Task
query = "Which two phrase in the first paragraph suggests that the writer did not have good eyesight"
answerqns({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'Do you want thicker lenses for your glasses?'}

In [7]:
# QA Task
query = "List two actions which showed that the writer’s mother approved of the writer helping out with the dumplings in the kitchen."
answerqns({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'Mother smiled at the writer and said, “Yes, that’s a good girl.”'}

In [8]:
# Classification Task
query = "Mother agreed that the dumplings the writer made did not look beautiful. True or False?"
answerqns({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'True'}

In [10]:
# Classification Task
query = "The writer was bullied in school because she was always sick. True or False?"
answerqns({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'False'}

## Summary thoughts
1. As demonstrated above, an open-sourced pre-trained google t5 LLM (11B parameters) can effectively provide answers to comprehension exam questions (abet at PSLE for now), using customized prompting templates.
2. Such LLM can be quickly reprogrammed to perform different task using various prompts. Given this flexibility, the possibilities for the developers are unlimited.
3. What I will like to build is a personalized assistant that can read through all emails and provide summarization assistance. Using a more intuitive dialogue UI, LLM assistants can scan private emails and provide concise answers to directed queries, all in the safety of a trusted compute environment.
4. From a knowledge management/documentation perspective, it can also help officers improve productivity especially during handovers, especially when incumbent officers need to get up to speed on the details of pertinent terms and conditions regarding the history of the project.
5. As shared in my [previous post](http://keith.ml/lognmetrics.html), in my past life I used to spend at least 2 hours everyday reading and writing emails, these 2 hours can be effectively cut down by half to 1 hour instead.
6. Excited that Meta Research has offered interested parties access to their 65B [LLAMA](https://github.com/facebookresearch/llama) model. Hopefully that will be for another post.

In [11]:
## Lastly why not lets ask the model about itself
llm.generate(["What is the future of large language models?","Will they replace humans?"]).to_dict()['generations']

[[{'text': 'The future of large language models is to be used in a variety of applications, including speech recognition, speech synthesis, and natural language processing.',
   'generation_info': None}],
 [{'text': 'no', 'generation_info': None}]]